# Challenge 4: Telemetry to the Rescue

## Find crash in telemetry data

You need to design a SQL query that returns the `car_number`, `driver_name` and the average `brake` & `speed` for the full second just before the crash. You should have the timestamp of the crash from the previous challenge.

> **Keep in mind that the timestamp is in local time (UTC + 2), whereas the telemetry data uses UTC.**

In [ ]:
%%bigquery telemetry_during_crash
# Find the crash in telemetry data that is loaded into BQ. Add your SQL query below



In [ ]:
print(telemetry_during_crash.to_markdown())

## Find the drivers

Now we have the filtered & aggregated data, let's ask Gemini to tell us which two drivers were involved in the crash.

Design a prompt that determines **which drivers** were involved and **why the model thinks that**.

> Note that we're *appending the result of the query from the previous cell to your prompt*, so you can refer to that.

In [ ]:
from google import genai
from IPython.display import Markdown

PROJECT_ID = "-" # Add your project ID here
LOCATION = "us-central1"

In [9]:
# ADD YOU PROMPT HERE
# Keep the `{data}` segment at the end. There is where your SQL will be inserted
prompt = """

Data:
{data}
"""

In [ ]:

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents = prompt.format(data=telemetry_during_crash.to_markdown())
)

Markdown(response.text)